In [1]:
using Pkg
if Sys.KERNEL == :Linux
    environment = "../stoch_growth_model_server"
elseif Sys.KERNEL == :Darwin
    environment = "AAA - Honours Projec/stoch_growth_model_for_alice/"
end
    
Pkg.activate("$environment")

  Activating project at `~/stoch_growth_model_for_alice/stoch_growth_model_server`


In [ ]:
# Pkg.add(PackageSpec(name="Catalyst", version="15.0.4"))
# Pkg.add(PackageSpec(name="DifferentialEquations", version="7.16.1"))
# Pkg.add(PackageSpec(name="DataFrames", version="1.7.0"))
# Pkg.add(PackageSpec(name="CSV", version="0.10.15"))
# Pkg.add("Plots")


In [2]:
using Catalyst
using DifferentialEquations
using DataFrames
using CSV
using Plots

In [14]:
include("model_eqs_tutorial.jl")
include("model_div_tutorial.jl")
include("../src/setup_funcs.jl")
params = getPars("uM", "NatComms"; abx=0.0)
u0 = getX0(gm_divC; ss=false, parametrization="NatComms")

18-element Vector{Pair{Num, Float64}}:
   a(t) => 10.0
  si(t) => 0.0
  mm(t) => 0.0
  mt(t) => 0.0
  mq(t) => 0.0
  mr(t) => 0.0
  cm(t) => 0.0
  ct(t) => 0.0
  cq(t) => 0.0
  cr(t) => 0.0
  em(t) => 0.0
  et(t) => 0.0
   q(t) => 0.0
   r(t) => 10.0
 zmm(t) => 0.0
 zmt(t) => 0.0
 zmq(t) => 0.0
 zmr(t) => 0.0

In [17]:
println("Solving ODE system")
prob = ODEProblem(gm_divC, u0, (0.0, 1e3), params)
sol = solve(prob, Rodas5(); saveat=1.0)
println("Simulation complete")


Solving ODE system
Simulation complete


In [19]:

df = DataFrame(sol)
CSV.write("model_div_results.csv", df)

plt = plot(sol;
    vars=[:a, :si, :r, :em, :et, :q],
    lw=2,
    xlabel="Time",
    ylabel="Concentration (µM)",
    legend=:outerright,
    title="Model DIV ODE Simulation"
)
savefig("model_div_ODE_results.png")
println("Saved outputs: model_div_results.csv and model_div_ODE_results.png")

┌ Warning: To maintain consistency with solution indexing, keyword argument vars will be removed in a future version. Please use keyword argument idxs instead.
│   caller = ip:0x0
└ @ Core :-1


Saved outputs: model_div_results.csv and model_div_ODE_results.png


In [ ]:
# To save changes each time
# in a server terminal: 
# git add . 
# git commit -m "message"
# git push


# If working somewhere else do: git pull